# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

In [3]:
value_dt = Date(2, 8, 2007)
settle_dt = value_dt.add_weekdays(1)

## Build Ibor Curve

In [4]:
dc_type = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(value_dt, "1D", 0.0500, dc_type); depos.append(depo)
depo = IborDeposit(settle_dt, "1D", 0.0500, dc_type); depos.append(depo)

fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0502,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0502,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0501,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0502,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0501,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
step_in_dt = value_dt.add_weekdays(1)

In [6]:
maturity_3yr = value_dt.next_cds_date(36)
maturity_5yr = value_dt.next_cds_date(60)
maturity_7yr = value_dt.next_cds_date(84)
maturity_10yr = value_dt.next_cds_date(120)

### Heterogeneous Curves

In [7]:
# Move into the local folder of notebook
%pwd
%cd notebooks/products/credit/
%pwd


[WinError 3] The system cannot find the path specified: 'notebooks/products/credit/'
C:\Users\domok\Dropbox\Desktop\RESEARCH_DB\FinancePy\Code\financepy-git\notebooks\products\credit


'C:\\Users\\domok\\Dropbox\\Desktop\\RESEARCH_DB\\FinancePy\\Code\\financepy-git\\notebooks\\products\\credit'

In [8]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [9]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40

In [10]:
cdsIndex = CDSIndexPortfolio()

In [11]:
spd3Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_3yr, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_5yr, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_7yr, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_10yr, heteroIssuerCurves)

In [12]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.67855145586222
Homogeneous curve 5Y: 35.537520216151265
Homogeneous curve 7Y: 49.008062953328185
Homogeneous curve 10Y: 61.407349264326484


In [13]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Tranches

In [14]:
trancheMaturity = maturity_5yr
tranche1 = CDSTranche(value_dt, trancheMaturity, 0.00, 0.03)
tranche2 = CDSTranche(value_dt, trancheMaturity, 0.03, 0.06)
tranche3 = CDSTranche(value_dt, trancheMaturity, 0.06, 0.09)
tranche4 = CDSTranche(value_dt, trancheMaturity, 0.09, 0.12)
tranche5 = CDSTranche(value_dt, trancheMaturity, 0.12, 0.22)
tranche6 = CDSTranche(value_dt, trancheMaturity, 0.22, 0.60)
tranche7 = CDSTranche(value_dt, trancheMaturity, 0.00, 0.60)
tranche8 = CDSTranche(value_dt, trancheMaturity, 0.00, 1.00)

In [15]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [16]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [17]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "k_1", "k_2", "SPD(BPS)"))
for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_dt,homoIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, num_points, tranche.k1*100, tranche.k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts       k_1       k_2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.466884
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.466884
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   908.968772
                    FinLossDistributionBuilder.LHP    50   0.00000   3.00000   914.784522
              FinLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.596331
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.596331
               FinLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   239.285369
                    FinLossDistributionBuilder.LHP    50   3.00000   6.00000   226.821508
              FinLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.118666
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   9.00000   102.118666
         

               FinLossDistributionBuilder.GAUSSIAN    50   0.00000  60.00000    60.175832
                    FinLossDistributionBuilder.LHP    50   0.00000  60.00000    59.212721
              FinLossDistributionBuilder.RECURSION    50   0.00000 100.00000    35.374671
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000 100.00000    35.374671
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000 100.00000    35.942993
                    FinLossDistributionBuilder.LHP    50   0.00000 100.00000    35.374663


## Heterogeneous Portfolio Results

In [18]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "k_1", "k_2", "SPD(BPS)"))

for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_dt,heteroIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, num_points, tranche.k1*100, tranche.k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts       k_1       k_2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.984779
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.916608
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   983.655339
                    FinLossDistributionBuilder.LHP    50    0.00000   3.00000   915.154077
              FinLossDistributionBuilder.RECURSION    50    3.00000   6.00000   230.901641
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   230.993770
               FinLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   230.457304
                    FinLossDistributionBuilder.LHP    50    3.00000   6.00000   226.820737
              FinLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.191640
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    6.00000   9.00000    87.249549

                    FinLossDistributionBuilder.LHP    50   22.00000  60.00000     0.417912
              FinLossDistributionBuilder.RECURSION    50    0.00000  60.00000    59.212818
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000  60.00000    59.212818


               FinLossDistributionBuilder.GAUSSIAN    50    0.00000  60.00000    60.116572
                    FinLossDistributionBuilder.LHP    50    0.00000  60.00000    59.212781
              FinLossDistributionBuilder.RECURSION    50    0.00000 100.00000    35.374318
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000 100.00000    35.374318
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000 100.00000    35.907133
                    FinLossDistributionBuilder.LHP    50    0.00000 100.00000    35.374301


Copyright (c) 2020 Dominic O'Kane